In [ ]:
import json 
from gufe.tokenization import JSON_HANDLER
import numpy as np
import os 
import shutil
from pathlib import Path

for this dataset, we know we have 3 replicates run in serial for each leg. We want to manipulate the data so that it is equivalent to the output if we re-ran this dataset with each leg run in parallel, with the following directory structure:

```
results/
  transformations_0/
      rbfe_lig_ejm_31_complex_lig_ejm_42_complex/
          shared_[hashA]_attempt_0/
      rbfe_lig_ejm_31_complex_lig_ejm_42_complex.json
  transformations_1/
      rbfe_lig_ejm_31_complex_lig_ejm_42_complex/
          shared_[hashB]_attempt_0/
      rbfe_lig_ejm_31_complex_lig_ejm_42_complex.json
  transformations_2/
      rbfe_lig_ejm_31_complex_lig_ejm_42_complex/
          shared_[hashC]_attempt_0/
      rbfe_lig_ejm_31_complex_lig_ejm_42_complex.json
```

In [ ]:
def load_json(fpath):
    return json.load(open(fpath, 'r'), cls=JSON_HANDLER.decoder)

def dump_json(data, fpath):
    with open(fpath, "w") as f:
        json.dump(data, f, cls=JSON_HANDLER.encoder)

In [ ]:
orig_dir = Path("results/")
new_dir = Path("results_parallel/")

In [ ]:
leg_names = []
for name in os.listdir(orig_dir):
    if name.endswith(".json"):
        continue
    leg_names.append(name)
leg_names

In [ ]:
! rm -rf $new_dir
for leg in leg_names:
    json_data = load_json(orig_dir/f"{leg}.json")
    srckey_to_protocol = {}
    srckey_to_unit_results = {}
    srckey_to_estimate = {}
    ## collect results on a per-replicate basis
    for k in json_data['protocol_result']['data']:    
        rep_source_key = json_data['protocol_result']['data'][k][0]['source_key']
        
        # keep only the data for this replicate
        rep_result = json_data['protocol_result'].copy()
        rep_result['data']={k:json_data['protocol_result']['data'][k]}
        srckey_to_protocol[rep_source_key] = rep_result

        # pull just the estimate value so we can put it at the top of the output
        srckey_to_estimate[rep_source_key] = rep_result['data'][k][0]['outputs']['unit_estimate']
        
    for k in json_data['unit_results']:
        rep_source_key = json_data['unit_results'][k]['source_key']

        rep_unit_result = json_data['unit_results'].copy()
        rep_unit_result = {k: json_data['unit_results'][k]}
        srckey_to_unit_results[rep_source_key] = rep_unit_result
    
    assert srckey_to_protocol.keys() == srckey_to_unit_results.keys()
    
    ## write to the new directory
    for n, sk in enumerate(sorted(srckey_to_protocol.keys())):
        rep_dir = new_dir/f"replicate_{n}"
        os.makedirs(rep_dir/leg)
    
        # build up the data for this replicate
        replicate_data = {'estimate': srckey_to_estimate[sk],
                          'uncertainty': np.std(srckey_to_estimate[sk]),
                          'protocol_result': srckey_to_protocol[sk],
                          'unit_results': srckey_to_unit_results[sk]}
    
        # write!
        dump_json(replicate_data, rep_dir/f"{leg}.json")
        working_dir_name = f"shared_{sk}_attempt_0"
        ## TODO: make this work for arbitrary number of attempts 
        # os.symlink(orig_dir/leg/working_dir_name, rep_dir/leg/working_dir_name)
        shutil.copytree(orig_dir/leg/working_dir_name, rep_dir/leg/working_dir_name)


## check that objects reload correctly

In [ ]:
import openfe

In [ ]:
json_reloaded = load_json("results_parallel/replicate_0/easy_rbfe_lig_ejm_31_solvent_lig_ejm_47_solvent.json")

In [ ]:
json_reloaded['estimate'], json_reloaded['uncertainty']

In [ ]:
pr_reloaded = openfe.ProtocolResult.from_dict(json_reloaded['protocol_result'])

In [ ]:
pr_reloaded.data

In [ ]:
first_pur_key = list(json_reloaded['unit_results'].keys())[0]
pur_reloaded = openfe.ProtocolUnit.from_dict(json_reloaded['unit_results'][first_pur_key])

In [ ]:
pur_reloaded